<a href="https://colab.research.google.com/github/adeotti/sudoku-env/blob/main/M9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# python version 3.9.11 
%pip install numpy==1.25.2 tensorboard==2.17.0 torchrl==0.4.0 gymnasium==0.29.1 tensordict==0.4.0

from IPython.display import clear_output
def clear():
  clear_output(wait=False)

import math,sys 
import torch

clear()

In [2]:
easyBoard = torch.tensor([
    [0, 0, 0, 5, 3, 1, 0, 0, 0],
    [0, 0, 0, 0, 4, 0, 3, 0, 1],
    [1, 0, 0, 8, 0, 0, 0, 0, 0],
    [0, 0, 4, 0, 0, 5, 6, 0, 0],
    [0, 0, 3, 9, 0, 2, 1, 4, 0],
    [6, 1, 5, 0, 7, 0, 0, 9, 8],
    [0, 2, 0, 0, 9, 6, 0, 1, 0],
    [0, 5, 7, 2, 0, 8, 0, 0, 6],
    [0, 6, 1, 7, 5, 3, 0, 2, 4]])

solution = torch.tensor([
    [8, 4, 9, 5, 3, 1, 7, 6, 2],
    [5, 7, 2, 6, 4, 9, 3, 8, 1],
    [1, 3, 6, 8, 2, 7, 4, 5, 9],
    [2, 9, 4, 1, 8, 5, 6, 7, 3],
    [7, 8, 3, 9, 6, 2, 1, 4, 5],
    [6, 1, 5, 3, 7, 4, 2, 9, 8],
    [3, 2, 8, 7, 9, 5, 1, 6, 7],
    [4, 5, 7, 2, 1, 8, 9, 3, 6],
    [9, 6, 1, 7, 5, 3, 8, 2, 4]])

In [3]:
from dataclasses import dataclass

@dataclass(frozen=True)
class Board_specs:
  size: tuple = (9,9)
  low: int = 1
  high: int = 9

def Board():
  matrix = easyBoard
  return matrix

def Region_Split(matrix):
  # Extract the 3X3 regions
    ZERO = 0 ; CONST = 3 ; MAX_COLUMN_END = 12
    row_start = col_start = ZERO
    row_end = col_end = CONST
    subgrid = []
    while len(subgrid) < len(matrix) :
        subgrid.append([row[col_start:col_end] for row in matrix[row_start:row_end]])
        col_start += CONST
        col_end += CONST
        if col_end == MAX_COLUMN_END :
            col_start = ZERO
            col_end = CONST
            row_start += CONST
            row_end += CONST
    assert len(subgrid) == len(matrix)
    subgrid = [torch.concatenate(line) for line in subgrid]
    return subgrid

Game and Utility class 

In [4]:
class Game:
    def __init__(self,action = None):
        self.board = Board()
        self.action = action

        self.modifiableCells = []
        
        for i,x in enumerate(self.board):
            for y in range(Board_specs.high):
                if x[y] == 0:
                    self.modifiableCells.append((i,y))            

    def Updated_board(self):
        if self.action is not None:
            row,column,value = self.action
            if (row,column) in self.modifiableCells:
                self.board[row][column] = value     
        return self.board,self.action
    

class Util:
    @classmethod
    def Utilities(cls,gameinstance):
        if gameinstance is None :
            sys.exit(f"{cls.__name__} : No Game instance was provided")
        cls.upd_matrix,cls.action = gameinstance.Updated_board()

        cls.y_list = [[col[y_index] for col in cls.upd_matrix] for y_index in range(Board_specs.high)]
        cls.y = [line for line in torch.t(cls.upd_matrix)]
        
        cls.subgrid = Region_Split(cls.upd_matrix)
        cls.subgrid = [arr.tolist() for arr in cls.subgrid]

    @staticmethod
    def unique(lisst):
        return len(lisst) == len(set(lisst))

    @staticmethod
    def len_dif(lisst):
        return len(lisst) - len(set(lisst))

Reward Function, Heuristic and the end of game function

In [5]:
class boardConflicts: # Heuristic component

    previous_conflicts = 0

    def __init__(self,instance):
        Util.Utilities(instance)
        self.conflicts = 0
        self.previous_conflicts = 0
        self.reward_H = 0

    def count(self):
        for line in Util.upd_matrix:
            if not Util.unique(line) : self.conflicts += Util.len_dif(line)
        for col in Util.y_list:
            if not Util.unique(col) : self.conflicts += Util.len_dif(col)
        for submatrix in Util.subgrid:
            if not Util.unique(submatrix) : self.conflicts += Util.len_dif(submatrix)
        # Heuristic value application
        if self.conflicts < self.previous_conflicts:
            self.reward_H = 2
        boardConflicts.previous_conflicts = self.conflicts
        return self.reward_H
    

class rewardFunction:
    def __init__(self,instance):
        Util.Utilities(instance)
        self.action = Util.action
        self.matrix = Util.upd_matrix
        self.y_list = Util.y_list
        self.reward = 0

    def rewardReturns(self):
        if self.action is None :
            sys.exit(f"{self.__class__.__name__} : Action is None")

        # x
        x = self.matrix[self.action[0]]
        assert len(x) == Board_specs.high
        self.reward += 9 if (Util.len_dif(x) == 0) else - Util.len_dif(x)

        # y
        y_index = self.action[1]
        y = self.y_list[y_index]
        assert len(y) == Board_specs.high
        self.reward += 9 if (Util.len_dif(y) == 0) else -Util.len_dif(y)

        # region
        n = int(math.sqrt(Board_specs.high))
        ix,iy = (self.action[0]//n)* n , (self.action[1]//n)* n
        region = torch.flatten(self.matrix[ix:ix+n , iy:iy+n])
        assert len(region) == Board_specs.high

        self.reward += 9 if (Util.len_dif(region) == 0) else  -Util.len_dif(region)
        return self.reward


def gameEnd(instance):
    Util.Utilities(instance)
    matrix = Util.upd_matrix
    return torch.equal(solution,matrix)

Environment

In [6]:
from torchrl.envs import EnvBase
from torchrl.data import BoundedTensorSpec,CompositeSpec
from tensordict import TensorDictBase,TensorDict

class environment(EnvBase):
    def __init__(self):
        super().__init__()

        self.action = None
        self.game = Game(self.action)
        self.updatedBoard,_ = self.game.Updated_board()

        self.action_spec = BoundedTensorSpec(
            low=torch.tensor([0,0,1]),
            high=torch.tensor([9,9,9]),
            shape=torch.Size([3,]),
            dtype=torch.int
        )

        self.observation_format = BoundedTensorSpec(
            low=1.0,
            high=9.0,
            shape=(easyBoard).unsqueeze(0).shape,
            dtype=torch.float32
        )
        self.observation_spec = CompositeSpec(observation = self.observation_format)

    def _step(self,tensordict) -> TensorDictBase :
        self.action = tensordict["action"].tolist()[0] # original shape -> tensor([[0, 1, 2]])
        self.updated,_ = Game(self.action).Updated_board()

        self.game = Game(self.action)
        reward = rewardFunction(self.game).rewardReturns()

        output = TensorDict(
            {
                "observation" : self.updatedBoard.clone().detach().unsqueeze(0).float(),
                "reward" : torch.tensor(reward),
                "done" : gameEnd(self.game)
            }
        )
        return output

    def _reset(self,tensordict,**kwargs) -> TensorDictBase :
        output = TensorDict(
            {
                "observation" :  self.updatedBoard.clone().detach().unsqueeze(0).float()
                }
        )
        return output

    def _set_seed(self):
        pass

In [7]:
from tqdm import tqdm
from collections import deque

import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical

from tensordict.nn import TensorDictModule

from torchrl.modules import ValueOperator,ProbabilisticActor
from torchrl.objectives.value import GAE
from torchrl.objectives import ClipPPOLoss

from torchrl.collectors import SyncDataCollector
from torchrl.data import ReplayBuffer,SamplerWithoutReplacement,LazyTensorStorage

from torch.utils.tensorboard import SummaryWriter

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hypers
l_rate = 0.01
sdg_momentum = 0.9

frames =  10             # number of steps
sub_frame = 5              # for the most inner loop of the training step
total_frames = 20      # maximum steps
epochs = 10

gamma = 0.80
lmbda = 0.99

env = environment()
dummy_observation = env._reset(None)["observation"] 

@torch.no_grad()
def weights_init(w):
  if isinstance(w,(nn.Conv2d,nn.LazyConv2d,nn.LazyLinear)):
    nn.init.kaiming_uniform(w.weight,mode="fan_in",nonlinearity="relu")
    if w.bias is not None : nn.init.zeros_(w.bias)

def networkInit(network : nn.Module):
  network.to(device)
  network.forward(dummy_observation)
  network.apply(weights_init)
  return network

In [ ]:
class Mask: 
  # Altering the softmax distribution so value in [x,y,value] != 0 
  # while keeping track of the gradient during backpropagation
  def __init__(self):
    self.newValue = -float("inf")

  def apply(self,tensor : torch.FloatTensor):
    self.mask = torch.zeros_like(tensor,dtype=torch.bool)
    self.mask[-1,-1,0] = True
    tensor = tensor.masked_fill(mask=self.mask,value=self.newValue)
    return tensor


class ActorNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.size = 81
    self.outputShape = 27 # 3*9 = 27 haha
    self.outputReshaped = (3,9)
    self.mask = Mask()

    self.input_layer = nn.LazyLinear(81)
    self.flat = nn.Flatten()
    self.dense_one = nn.LazyLinear(self.size)
    self.dense_two = nn.LazyLinear(self.size)
    self.output = nn.LazyLinear(self.outputShape)

  def forward(self,x):
    x = self.flat(x)
    x = F.relu(self.input_layer(x))
    x = F.relu(self.dense_one(x))
    x = F.relu(self.dense_two(x))
    x = F.relu(self.output(x))
    x = torch.unflatten(x,-1,(self.outputReshaped))
    x = self.mask.apply(x)
    return F.softmax(x,-1)

Actor = networkInit(network=ActorNetwork()) 

Policy = TensorDictModule(
  module=Actor, 
  in_keys=["observation"],
  out_keys=["probs"]
)

PolicyModule = ProbabilisticActor(
  module=Policy ,
  spec=env.action_spec,in_keys=["probs"],
  distribution_class = Categorical,
  return_log_prob = True
)

In [10]:
Memory = ReplayBuffer(
    storage=LazyTensorStorage(max_size=frames),
    sampler=SamplerWithoutReplacement()
)

Collector = SyncDataCollector(
    create_env_fn=env,
    policy=PolicyModule,
    frames_per_batch=frames,
    total_frames=total_frames
)
Collector.rollout()
clear()

In [11]:
class ValueNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.size = 81
    self.input_layer = nn.LazyLinear(self.size)
    self.flat = nn.Flatten()
    self.dense_one = nn.LazyLinear(self.size)
    self.dense_two = nn.LazyLinear(self.size)
    self.output = nn.LazyLinear(1)

  def forward(self,x):
    x = self.flat(x)
    x = F.relu(self.input_layer(x))
    x = F.relu(self.dense_one(x))
    x = F.relu(self.dense_two(x))
    return self.output(x)

Critic = networkInit(network=ValueNetwork())

ValueModule = ValueOperator(
  module= Critic,
  in_keys=["observation"]
)

Advantage = GAE(
  gamma=gamma,
  lmbda=lmbda,
  value_network=ValueModule,
  average_gae=True,
  device=device
)

In [12]:
loss = ClipPPOLoss(
    actor_network=PolicyModule,
    critic_network=ValueModule
)

optimizer = torch.optim.SGD(
    params=loss.parameters(),
    lr=l_rate,
    momentum=sdg_momentum 
)

Training loop

In [15]:
class Training:
    def __init__(self):
        self.policy = ActorNetwork()
        self.collector = Collector
        self.memory = Memory
        self.valuemodule = ValueModule
        self.advantage = Advantage
        self.lossfunction = loss
        self.optimizer = optimizer
        self.epochs = epochs

    def save_logs(self):
            log_dir = "trainingData/new/"  
            self.writer = SummaryWriter(log_dir)

    def save_weight(self):
        path = "trainingData/new/actor_100k.pth"  
        torch.save(self.policy.state_dict(),path)
    
    def fullyTrainedmodel(self):
        path = "trainingData/new/fullyTrainedmodel.pth"
        torch.save(self.policy.state_dict(),path)

    def train(self,start : bool = None):
        if start:
            bestReward = -10
            rewardHistory = deque(maxlen = 10)
            self.save_logs()
            for i,data_tensordict in tqdm(enumerate(self.collector),total = total_frames/frames):

                for e in range(self.epochs):
                    dat = self.advantage(data_tensordict)
                    dat["advantage"] = dat["advantage"].unsqueeze(-1)
                    self.memory.extend(dat)

                    for n in range(total_frames//sub_frame):
                        subdata = Memory.sample(sub_frame)
                        loss_val = self.lossfunction(subdata.to(device))
                        loss_value = (loss_val["loss_objective"] + loss_val["loss_critic"] + loss_val["loss_entropy"])
                        loss_value.backward()
                        self.optimizer.step()
                        self.optimizer.zero_grad()

                self.writer.add_scalar("main/batch_number",i)
                self.writer.add_scalar("main/Advantage",dat["advantage"][0].item())
                self.writer.add_scalar("main/Loss_sum",loss_value.item())
                self.writer.add_scalar("main/reward",data_tensordict["next"]["reward"][0].mean().item())
                self.writer.add_scalar("main/reward_advantage",dat["next"]["reward"][0].mean().item())
                self.writer.add_scalar("loss/Loss_entropy",loss_val["loss_entropy"].item())
                self.writer.add_scalar("loss/Loss_critic",loss_val["loss_critic"].item())
                self.writer.add_scalar("loss/Loss_objective",loss_val["loss_objective"].item())

                currentReward = data_tensordict["next"]["reward"][0].mean()
                rewardHistory.append(currentReward)
                averageReward = sum(rewardHistory)/len(rewardHistory)

                if averageReward > bestReward:
                    self.save_weight()
                    bestReward = averageReward

            self.fullyTrainedmodel() # saving at the end of the training just for safety 

In [16]:
Training().train(start=True)

  0%|          | 0/2.0 [00:00<?, ?it/s]
